In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# This file contains all the main external libs we'll use
from fastai.imports import *

In [ ]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

from sklearn.metrics import confusion_matrix

In [ ]:
os.listdir('dogbreeds/')

In [ ]:
PATH = 'dogbreeds/'
labels = f'{PATH}labels.csv'

sz = int(244*1.3) # Can start from 64, 128, 256, etc.
bs = 58 # Decrease if facing memory error
n =  len(list(open(labels)))-1

cv_idx = get_cv_idxs(n)

In [ ]:
def get_data(sz,bz):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on,max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH,'train', f'{PATH}labels.csv', test_name='test',
                                        tfms=tfms,bs=bs,val_idxs=cv_idx,suffix='.jpg')
    return data

In [ ]:
arch = resnet50
data = get_data(sz,bs)
learn = ConvLearner.pretrained(arch,data,precompute=True)

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
for i in range(4):
    learn.fit(1e-1,1,cycle_len=1)
    learn.save(f"Resnet50_{sz}_epoch{i}")

In [ ]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0],4,replace=False)
def rand_by_correct(is_correct): return rand_by_mask((data.val_y==preds)==is_correct)

In [ ]:
def plot(images,title):
    """Only plot list of images horizontally"""
    f, plots = plt.subplots(1,len(images), figsize=(10,8))
    for i,j,k in zip(plots,images,title):
        i.axis('Off')
        i.set_title(k)
        i.imshow(plt.imread(f'{PATH}\\{j}'))

def plot_mask(mask,title):
    print(title)
    indices = mask
    title_probs = [max_probs[i] for i in indices]
    plot([data.val_ds.fnames[i] for i in indices],title_probs)

# Prepping for Kaggle Submission

In [ ]:
pd.set_option('display.float_format', lambda x: '%.17f' % x)

In [ ]:
a = pd.DataFrame([i[5:-4] for i in data.test_ds.fnames])
b = pd.DataFrame(np.exp(np.mean(log_preds,0)))
c = pd.concat((a,b),axis=1)
c.columns = ['id']+data.classes
c

In [ ]:
fileName = 'submission3.csv'
message = 'Your message goes here'
c.to_csv(fileName, index=False, header=True, decimal='.', sep=',', float_format='%.17f')

In [ ]:
!kaggle competitions submit -c dog-breed-identification -f {fileName} -m {message}